# Correction Vietnamese local strict spelling 

- Tích hợp hàm trích xuất danh từ riêng và số thông minh hơn.

# Libraries

In [1]:
import os
import re
import sys
import gc


import signal
import logging
import pandas as pd
import torch
import unicodedata
import difflib
from underthesea import ner


from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from huggingface_hub import snapshot_download


# Suppress Transformers Warning

In [2]:
logging.getLogger("transformers.generation.utils").setLevel(logging.ERROR)


# LOAD .env 

In [3]:
dotenv_path = os.path.join(os.getcwd(), "..", "envs", ".env")
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
else:
    load_dotenv()
    
print(f"dotenv_path: {dotenv_path}")


dotenv_path: /home/guest/Projects/DSC2025/BAN/preprocess/../envs/.env


In [4]:
HF_TOKEN = (
    os.getenv("HUGGING_FACE_TOKEN") or
    os.getenv("HUGGINGFACE_TOKEN") or
    os.getenv("HF_TOKEN") or
    os.getenv("HUNGGING_FACE_TOKEN") or
    None
)

if HF_TOKEN:
    print("OK!")
else:
    print("Add HF key to .env")


OK!


In [5]:
public_test_correction_folder = "./vihallu_public_test_correction"
os.makedirs(public_test_correction_folder, exist_ok=True)
print(f"public_test_correction_folder_path: {public_test_correction_folder}")

debug_correction_villua_folder = "./debug_vihallu_public_test_correction"
os.makedirs(debug_correction_villua_folder, exist_ok=True)
print(f"debug_correction_villua_folder_path: {debug_correction_villua_folder}")


public_test_correction_folder_path: ./vihallu_public_test_correction
debug_correction_villua_folder_path: ./debug_vihallu_public_test_correction


# Config

In [6]:
MODEL = "VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain"
INPUT_CSV = "./vihallu-public-test.csv"
OUTPUT_CSV =  os.path.join(public_test_correction_folder, "fixed-vihallu-public-test_final.csv")
COLUMNS_TO_FIX = ['context', 'prompt', 'response']

# <<< THAY ĐỔI: Thêm BATCH_SIZE để xử lý hàng loạt >>>
BATCH_SIZE = 4 # Bạn có thể điều chỉnh tùy theo VRAM
MAX_INPUT_LENGTH = 1024
WORD_COUNT_TOLERANCE = 3
ACCEPT_SIMILARITY_THRESHOLD = 0.88
LENGTH_CHANGE_ALLOWED_RATIO = 0.15 
STRICT_BASE_SIMILARITY      = 0.96
LENIENT_BASE_SIMILARITY     = 0.93      


DEBUG_LOG_CSV =  os.path.join(debug_correction_villua_folder, "debug_corrections_final.csv")
SAMPLE_DEBUG_N = 20


In [7]:
FEW_SHOTS = [
    ("Nguồn thhu chinh của thuê truc tiếp la nhủg ai?", "Nguồn thu chính của thuế trực tiếp là những ai?"),
    ("Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision là gì z?", "Ý nghĩa của việc tổ chức cuộc thi Intervision là gì?"),
    ("thủ đô ha nội là trung tâm văn hóa lớn nhất nc ta", "Thủ đô Hà Nội là trung tâm văn hóa lớn nhất nước ta."),
    ("...không đứng đầu thế giới về dự trữ ngoại tệ?", "...không đứng đầu thế giới về dự trữ ngoại tệ?"),
    ("Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?", "Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?"),
    ("Mô hình encoder-decoder sử dụng trong bài này là T5.", "Mô hình encoder-decoder sử dụng trong bài này là T5."),
]

for idx, few_shot in enumerate(FEW_SHOTS):
    print(f"index: {idx}\n+ Original: {few_shot[0]}\n-> Correction: {few_shot[1]}\n")


index: 0
+ Original: Nguồn thhu chinh của thuê truc tiếp la nhủg ai?
-> Correction: Nguồn thu chính của thuế trực tiếp là những ai?

index: 1
+ Original: Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision là gì z?
-> Correction: Ý nghĩa của việc tổ chức cuộc thi Intervision là gì?

index: 2
+ Original: thủ đô ha nội là trung tâm văn hóa lớn nhất nc ta
-> Correction: Thủ đô Hà Nội là trung tâm văn hóa lớn nhất nước ta.

index: 3
+ Original: ...không đứng đầu thế giới về dự trữ ngoại tệ?
-> Correction: ...không đứng đầu thế giới về dự trữ ngoại tệ?

index: 4
+ Original: Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?
-> Correction: Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?

index: 5
+ Original: Mô hình encoder-decoder sử dụng trong bài này là T5.
-> Correction: Mô hình encoder-decoder sử dụng trong bài này là T5.



# Device info

In [8]:
print("CUDA available:", torch.cuda.is_available())
print("\n=== CONFIG: ===")
print(f"Model: {MODEL}")
print(f"INPUT CSV: {INPUT_CSV}")
print(f"OUTPUT CSV: {OUTPUT_CSV}")
print(f"DEBUG_LOG_CSV: {DEBUG_LOG_CSV}")

print(f"\n=== Columns to fix: {COLUMNS_TO_FIX} ===")
print(f"MAX_INPUT_LENGTH: {MAX_INPUT_LENGTH}")
print(f"WORD_COUNT_TOLERANCE: {WORD_COUNT_TOLERANCE}")
print(f"SMAMPLE_DEBUG_N:{SAMPLE_DEBUG_N}")

print("\n=== DEBUG: Thresholds (Final Optimized) ===")
print(f"STRICT (prompt, response): BASE_SIMILARITY >= {STRICT_BASE_SIMILARITY}")
print(f"LENGTH_CHANGE_ALLOWED_RATIO: {LENGTH_CHANGE_ALLOWED_RATIO}")
print(f"LENIENT (context): BASE_SIMILARITY >= {LENIENT_BASE_SIMILARITY}")
print(f"GENERAL: ACCEPT_SIMILARITY >= {ACCEPT_SIMILARITY_THRESHOLD}")
print("====================================\n")


CUDA available: True

=== CONFIG: ===
Model: VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain
INPUT CSV: ./vihallu-public-test.csv
OUTPUT CSV: ./vihallu_public_test_correction/fixed-vihallu-public-test_final.csv
DEBUG_LOG_CSV: ./debug_vihallu_public_test_correction/debug_corrections_final.csv

=== Columns to fix: ['context', 'prompt', 'response'] ===
MAX_INPUT_LENGTH: 1024
WORD_COUNT_TOLERANCE: 3
SMAMPLE_DEBUG_N:20

=== DEBUG: Thresholds (Final Optimized) ===
STRICT (prompt, response): BASE_SIMILARITY >= 0.96
LENGTH_CHANGE_ALLOWED_RATIO: 0.15
LENIENT (context): BASE_SIMILARITY >= 0.93
GENERAL: ACCEPT_SIMILARITY >= 0.88



In [9]:
if not os.path.exists(INPUT_CSV): 
    raise FileNotFoundError(f"Không tìm thấy file '{INPUT_CSV}'.")

df = pd.read_csv(INPUT_CSV)
df_global = df


In [10]:
processed_indices = set()
if os.path.exists(OUTPUT_CSV):
    print(f"Found existing output file at {OUTPUT_CSV}. Resuming process.")
    try:
        df_processed = pd.read_csv(OUTPUT_CSV)
        # File output của chúng ta có cột 'index' do hàm save tạo ra
        if 'index' in df_processed.columns:
            processed_indices = set(df_processed['index'])
            print(f"Loaded {len(processed_indices)} completed rows. Skipping them.")
            
            original_row_count = len(df)
            df = df[~df.index.isin(processed_indices)]
            print(f"Resuming with {len(df)} remaining rows out of {original_row_count}.")
    except pd.errors.EmptyDataError:
        print("Output file is empty. Starting from the beginning.")
    except Exception as e:
        print(f"Could not read processed rows from {OUTPUT_CSV}. Starting from scratch. Error: {e}")


Found existing output file at ./vihallu_public_test_correction/fixed-vihallu-public-test_final.csv. Resuming process.
Loaded 32 completed rows. Skipping them.
Resuming with 968 remaining rows out of 1000.


In [11]:
# bộ chứa cho các row đã được xử lý (chỉ những row này sẽ được lưu vào OUTPUT_CSV)
processed_rows = []
processed_indices = set()
# STOP flag (sẽ được set True bởi signal handler)
stop_requested = False


# hàm lưu hiện trạng (chỉ lưu các cột index, context, prompt, response, predict_label)
def save_processed_rows_and_exit():
    if not processed_rows:
        print("No new processed rows to save.")
        return

    # 1. Lấy dữ liệu từ bộ chứa
    output_df = pd.DataFrame(processed_rows)

    # 2. Đảm bảo đủ các cột cần thiết (Logic quan trọng từ code của bạn)
    required_cols = ["index", "id", "context", "prompt", "response", "predict_label"]
    for c in required_cols:
        if c not in output_df.columns:
            output_df[c] = ""
    
    # Sắp xếp lại các cột theo đúng thứ tự
    output_df = output_df[required_cols]

    # 3. Kiểm tra xem file đã tồn tại chưa để quyết định ghi header (Logic của tôi)
    file_exists = os.path.exists(OUTPUT_CSV) and os.path.getsize(OUTPUT_CSV) > 0

    # 4. Ghi file ở chế độ append (ghi tiếp)
    output_df.to_csv(OUTPUT_CSV, mode='a', header=not file_exists, index=False, encoding="utf-8-sig")
    
    print(f"Successfully saved/appended {len(output_df)} new rows to {OUTPUT_CSV}")
    processed_rows.clear() # Xóa list sau khi lưu


## Ctrl+C handler

In [12]:
def signal_handler(sig, frame):
    global stop_requested
    print("\n\n[Ctrl+C] Request received — will stop after current row and save processed rows...")
    stop_requested = True


if 'ipykernel' not in sys.modules and 'IPython' not in sys.modules:
    signal.signal(signal.SIGINT, signal_handler)
else:
    print("Detected IPython environment — relying on KeyboardInterrupt for interruption handling.")


Detected IPython environment — relying on KeyboardInterrupt for interruption handling.


## tokenizer/model setup -> set up 8-bit if not fallback to origin 16-bit

In [13]:
tokenizer_kwargs = {
    "use_fast": False, 
    "token": HF_TOKEN, 
    "padding_side": "left"
}


In [14]:
def load_tokenizer_local_or_remote(model_id):
    try:
        tok = AutoTokenizer.from_pretrained(model_id, local_files_only=True, **{k: v for k, v in tokenizer_kwargs.items() if k != "token"})
    except Exception:
        tok = AutoTokenizer.from_pretrained(model_id, **tokenizer_kwargs)
    if tok.pad_token is None: tok.pad_token = tok.eos_token
    return tok


In [15]:
def load_model_local_or_snapshot(model_id):
    cache_dir = os.path.expanduser("~/.cache/huggingface/hub")
    print(f"Cache huggingface path: {cache_dir}")
    loading_strategies = [
        # {"name": "8bit", "kwargs": {"quantization_config": BitsAndBytesConfig(load_in_8bit=True), "device_map": "auto", "token": HF_TOKEN, "cache_dir": cache_dir}},
        {"name": "4bit", "kwargs": {"quantization_config": BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16), "device_map": "auto", "token": HF_TOKEN, "cache_dir": cache_dir}},
        {"name": "bfloat16", "kwargs": {"torch_dtype": torch.bfloat16, "device_map": "auto", "token": HF_TOKEN, "cache_dir": cache_dir}},
    ]
    
    last_e = None
    for strat in loading_strategies:    
        try:
            print(f"Trying to load with config: {strat['name']}")
            m = AutoModelForCausalLM.from_pretrained(model_id, **strat["kwargs"])
            print(f"Successfully loaded with config: {strat['name']}")
            model_path = (
                snapshot_download(model_id, token=HF_TOKEN, cache_dir=cache_dir)
                if HF_TOKEN
                else cache_dir
            )
            return m, model_path
        except Exception as e:
            print(f"Failed with config: {strat['name']} - {type(e).__name__}: {e}")
            last_e = e
    raise RuntimeError("Cannot load model") from last_e


## Load tokenizer

In [16]:
print("Loading tokenizer...")
tokenizer = load_tokenizer_local_or_remote(MODEL)
tokenizer


Loading tokenizer...


LlamaTokenizer(name_or_path='VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain', vocab_size=32000, model_max_length=8192, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}
)

## Load model

In [17]:
print("Loading model...")
model, model_source = load_model_local_or_snapshot(MODEL)
model.eval()


Loading model...
Cache huggingface path: /home/guest/.cache/huggingface/hub
Trying to load with config: 4bit


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Successfully loaded with config: 4bit


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

In [18]:
try: 
    model_device = next(model.parameters()).device
except: 
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
print(f"Model device: {model_device}")
print(f"Model loaded from: {model_source}")


Model device: cuda:0
Model loaded from: /home/guest/.cache/huggingface/hub/models--VietnamAIHub--Vietnamese_llama2_7B_8K_SFT_General_domain/snapshots/12466d4619deed5fb5972760c082d7382151376c


# Utilities

## Remove diaccritics and punctuation

In [19]:
def remove_diacritics_and_punct(s: str):
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = unicodedata.normalize('NFC', s)
    s = re.sub(r'[^\w\s]', '', s)
    return s.lower()


## Nomalize spaces

In [20]:
def normalize_spaces(s: str): 
    return re.sub(r'\s+', ' ', s).strip()


# Calculate String Similarity

In [21]:
def string_similarity(a, b): 
    return difflib.SequenceMatcher(None, a, b).ratio()


## Extract number 

In [22]:
def extract_numbers(text: str):
    """Trích xuất tất cả các chuỗi số, bao gồm cả số có dấu phẩy và dấu chấm."""
    return set(re.findall(r'\d+[,.]?\d*', text))


## Extract proper nouns

In [23]:
def extract_proper_nouns(text: str):
    """
    Trích xuất các thực thể tên riêng (danh từ riêng) sử dụng underthesea NER.
    """
    # ner trả về list các tuple: (text, tag, start_pos, end_pos)
    # Chúng ta chỉ lấy phần text (phần tử đầu tiên)
    if not text or not isinstance(text, str):
        return set()
    try:
        # ner() trả về list các tuple: (entity_text, entity_type)
        entities = ner(text)
        # Chúng ta chỉ lấy phần text của các thực thể
        proper_nouns = {entity[0] for entity in entities}
        return proper_nouns
    except Exception:
        # Trả về set rỗng nếu underthesea gặp lỗi
        return set()


# LLM

## Prompt & Generation

In [24]:
def make_correction_prompt_llama2(original_text):
    fs_examples = "\n".join(
        [f"Văn bản gốc: \"{o}\"\nVăn bản đã sửa: \"{c}\"" for o, c in FEW_SHOTS]
    )
    system_prompt = (
        "Bạn là một robot hiệu đính MÁY MÓC và CẨN TRỌNG.\n"
        "**QUY TẮC SỐ 1 (QUAN TRỌNG NHẤT):** KHÔNG THAY ĐỔI Ý NGHĨA. "
        "Giữ nguyên từ đúng ngữ nghĩa (ví dụ: 'ngoại tệ'), giữ nguyên từ viết tắt không rõ (ví dụ: 'S.J').\n"
        "**QUY TẮC SỐ 2:** CHỈ SỬA lỗi chính tả, lỗi gõ phím, lỗi dấu thanh, lỗi viết hoa.\n"
        "**QUY TẮC SỐ 3:** Nếu văn bản đã đúng, lặp lại y hệt.\n"
        "**QUY TẮC SỐ 4:** TUYỆT ĐỐI không thay đổi, thêm, hoặc xóa bất kỳ con số hay tên riêng nào (ví dụ: '1995', 'Hà Nội', 'S.J')."
    )
    user_prompt = (
        "Các ví dụ:\n"
        f"{fs_examples}\n\n"
        "Sửa văn bản sau. Chỉ trả về văn bản đã sửa.\n"
        f"Văn bản gốc: \"{original_text}\""
    )
    return f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt} [/INST]\nVăn bản đã sửa: \""


## Generate from model

In [25]:
def generate_from_model(prompt, original_text_for_token_limit):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_INPUT_LENGTH)
    try: inputs = {k: v.to(model_device) for k, v in inputs.items()}
    except Exception: pass
    
    input_token_length = len(tokenizer.encode(original_text_for_token_limit, add_special_tokens=False))
    
    dynamic_min_new_tokens = int(input_token_length * 0.8)
    dynamic_max_new_tokens = int(input_token_length * 1.5) + 50
    
    gen_cfg = GenerationConfig(
        min_new_tokens=dynamic_min_new_tokens,
        max_new_tokens=dynamic_max_new_tokens,
        do_sample=False, 
        num_beams=1, 
        repetition_penalty=1.1, 
        early_stopping=False,
        pad_token_id=tokenizer.eos_token_id, 
        eos_token_id=tokenizer.eos_token_id
    )
    
    with torch.no_grad(): 
        out = model.generate(**inputs, generation_config=gen_cfg)
    full_decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    
    if "[/INST]" in full_decoded:
        response_part = full_decoded.split("[/INST]")[-1].strip()
        if response_part.lower().startswith("văn bản đã sửa:"):
            response_part = response_part[len("Văn bản đã sửa:"):].strip()
        corrected = response_part.split('\n')[0].strip()
        corrected = re.sub(r'^["\']|["\']$', '', corrected)
        
        if corrected.endswith('"'): 
            corrected = corrected[:-1]
        return corrected
    return ""


# Correction function

In [26]:
correction_cache = {}


In [27]:
def validate_correction(original_text, corrected_text, col_name, row_id, debug_print=False):
    """
    Hàm này KHÔNG gọi LLM. Nó chỉ nhận vào văn bản gốc và văn bản đã sửa,
    sau đó áp dụng các quy tắc guardrails để quyết định có chấp nhận thay đổi không.
    """
    accepted = False
    final_text = original_text
    reason = "not_changed"
    base_sim = 1.0

    if corrected_text and corrected_text != original_text:
        current_base_threshold = STRICT_BASE_SIMILARITY if col_name in ['prompt', 'response'] else LENIENT_BASE_SIMILARITY
        orig_words = normalize_spaces(original_text).split()
        corr_words = normalize_spaces(corrected_text).split()
        base_sim = string_similarity(remove_diacritics_and_punct(original_text), remove_diacritics_and_punct(corrected_text))

        passes_initial_checks = False
        if abs(len(orig_words) - len(corr_words)) > WORD_COUNT_TOLERANCE:
            reason = f"word_count_mismatch: original_words:{len(orig_words)} vs correction_word: {len(corr_words)}"
        elif base_sim < current_base_threshold:
            reason = f"base_similarity_too_low_{base_sim:.2f}_(threshold:{current_base_threshold})"
        elif len(original_text) > 0 and abs(len(corrected_text) - len(original_text)) / len(original_text) > LENGTH_CHANGE_ALLOWED_RATIO:
            reason = "length_changed_too_much"
        elif string_similarity(original_text, corrected_text) < ACCEPT_SIMILARITY_THRESHOLD:
            reason = f"low_similarity_{string_similarity(original_text, corrected_text):.2f}"
        elif original_text.endswith('?') and not corrected_text.endswith('?'):
            reason = "question_mark_removed"
        else:
            passes_initial_checks = True

        if passes_initial_checks:
            original_numbers = extract_numbers(original_text)
            corrected_numbers = extract_numbers(corrected_text)
            original_nouns = extract_proper_nouns(original_text)
            corrected_nouns = extract_proper_nouns(corrected_text)
            
            if original_numbers != corrected_numbers:
                reason = f"numbers_altered: {original_numbers.symmetric_difference(corrected_numbers)}"
                passes_initial_checks = False
            elif original_nouns != corrected_nouns:
                reason = f"proper_nouns_altered: {original_nouns.symmetric_difference(corrected_nouns)}"
                passes_initial_checks = False

        if passes_initial_checks:
            accepted = True
            reason = "accepted_change"
            final_text = corrected_text
            
    if debug_print:
        print("\\n--- DEBUG SAMPLE ---")
        print(f"### ID: {row_id} | COLUMN: {col_name}")
        print(f"+   ORIGINAL: \\n{original_text}")
        print(f"\\n+ CORRECTED (model returned): \\n{corrected_text}")
        if corrected_text and corrected_text != original_text:
            print("\\n--- METRICS & THRESHOLDS ---")
            print(f"  - Base Similarity: {base_sim:.4f}")
            print(f"  - Direct Similarity: {string_similarity(original_text, corrected_text):.4f}")
            print("\\n--- FACT CHECKING ---")
            print(f"  - Original Numbers (len: {len(extract_numbers(original_text))}): \\n{extract_numbers(original_text)}")
            print(f"  - Corrected Numbers (len: {len(extract_numbers(corrected_text))}): \\n{extract_numbers(corrected_text)}")
            print(f"  - Original Nouns (len: {len(extract_proper_nouns(original_text))}): \\n{extract_proper_nouns(original_text)}")
            print(f"  - Corrected Nouns (len: {len(extract_proper_nouns(corrected_text))}): \\n{extract_proper_nouns(corrected_text)}")
        else:
            print("\\n--- METRICS & THRESHOLDS ---")
            print("No change proposed by model.")
        print("\\n--- DECISION ---")
        print(f"RESULT: {'ACCEPTED' if accepted else 'REJECTED'}")
        if not accepted:
            print(f"REASON: {reason}")
        print(f"FINAL TEXT: \\n{final_text}")
        print("-" * 10)
        
    try:
        df_row = pd.DataFrame([{"id": row_id, "column": col_name, "original": original_text, "corrected_model": corrected_text, "final_text": final_text, "accepted": accepted, "reason": reason, "similarity": string_similarity(original_text, corrected_text) if corrected_text else 1.0, "base_similarity": base_sim}])
        log_header = not os.path.exists(DEBUG_LOG_CSV)
        df_row.to_csv(DEBUG_LOG_CSV, index=False, mode='a', header=log_header, encoding="utf-8-sig")
    except Exception as e:
        if debug_print: print(f"Warning: cannot write debug csv: {e}")
        
    return final_text, accepted


# Main Loop for correcting Vietnamese's spelling

## --- Giai đoạn 1: Thu thập & Generate ---

In [28]:
tasks_to_process = []
print("Phase 1: Collecting tasks and generating corrections...")

# Đảm bảo 'index' là một cột để dùng cho việc resume
if 'index' not in df.columns:
    df.reset_index(inplace=True)

for _, row in df.iterrows():
    # Bỏ qua các hàng đã xử lý trong các lần chạy trước
    if row['index'] in processed_indices:
        continue
    
    # Thu thập các tác vụ cần xử lý
    for col in COLUMNS_TO_FIX:
        original_text = str(row.get(col, '')).strip()
        if original_text:
            tasks_to_process.append({
                "df_index": row['index'],
                "id": row.get('id', row['index']),
                "column": col,
                "original_text": original_text
            })


Phase 1: Collecting tasks and generating corrections...


In [29]:
tasks_to_process[:10]


[{'df_index': 32,
  'id': 'ae7ea16b-f316-4c1d-b0d0-7f2a1465cdae',
  'column': 'context',
  'original_text': 'Birmingham là một phố chợ cỡ trung bình vào thời kỳ trung cổ, sau đó trở nên nổi bật ở tầm quốc tế trong thế kỷ 18 khi là trọng tâm trong Khai sáng Midlands rồi cách mạng công nghiệp. Trong cách mạng công nghiệp, Birmingham đi tiên phong trong các tiến bộ toàn cầu về phát triển khoa học, kỹ thuật, và kinh tế, sản sinh hàng loạt sáng kiến giúp đặt một phần nền tảng cho xã hội công nghiệp hiện đại. Đến năm 1791, Birmingham được ca ngợi là "thị trấn sản xuất đầu tiên trên thế giới". Thành phố có hồ sơ kinh tế đặc trưng, với hàng nghìn xưởng nhỏ đa dạng về các ngành nghề chuyên biệt và có kỹ năng cao, khuyến khích mức độ sáng tạo và sáng kiến cao khác thường, tạo ra cơ sở kinh tế đa dạng và linh hoạt cho giai đoạn thịnh vượng công nghiệp kéo dài cho đến cuối thế kỷ 20. Động cơ hơi nước công nghiệp được phát minh tại Birmingham, đây có lẽ là sáng kiến quan trọng nhất trong lịch sử An

In [ ]:
print(f"Found {len(tasks_to_process)} text fields to process in this run.")
generated_corrections = {} 

for i in tqdm(range(0, len(tasks_to_process), BATCH_SIZE), desc="Batch Generating", leave=True, dynamic_ncols=True):
    if stop_requested: break
    batch_tasks = tasks_to_process[i:i + BATCH_SIZE]
    original_texts_batch = [task['original_text'] for task in batch_tasks]
    prompts_batch = [make_correction_prompt_llama2(text) for text in original_texts_batch]

    inputs = tokenizer(prompts_batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_INPUT_LENGTH).to(model_device)
    max_len_in_batch = max(len(tokenizer.encode(t)) for t in original_texts_batch) if original_texts_batch else 0
    gen_cfg = GenerationConfig(max_new_tokens=int(max_len_in_batch * 1.5) + 50, do_sample=False, num_beams=1, pad_token_id=tokenizer.eos_token_id)
    
    with torch.no_grad(): 
        out = model.generate(**inputs, generation_config=gen_cfg)
    decoded_results = tokenizer.batch_decode(out, skip_special_tokens=True)

    for j, task in enumerate(batch_tasks):
        full_decoded = decoded_results[j]
        corrected_text = ""
        if "[/INST]" in full_decoded:
            response_part = full_decoded.split("[/INST]")[-1].strip()
            if response_part.lower().startswith("văn bản đã sửa:"): response_part = response_part[len("Văn bản đã sửa:"):].strip()
            corrected_text = re.sub(r'^["\']|["\']$', '', response_part.split('\\n')[0].strip())
        generated_corrections[(task['df_index'], task['column'])] = corrected_text


Found 2904 text fields to process in this run.


Batch Generating:   0%|          | 0/726 [00:00<?, ?it/s]

## --- Giai đoạn 2: Xác thực & Lưu ---

In [ ]:
print("\\nPhase 2: Validating results and updating DataFrame...")
corrected_ids = set()
if os.path.exists(DEBUG_LOG_CSV) and not processed_indices:
    os.remove(DEBUG_LOG_CSV)

try:
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Validating & Saving", leave=True, dynamic_ncols=True):
        if stop_requested: break
        
        current_index = row['index']
        if current_index in processed_indices: continue

        row_copy = row.copy()
        row_was_changed = False
        
        for col in COLUMNS_TO_FIX:
            original_text = str(row.get(col, '')).strip()
            if not original_text: continue
            
            corrected_text_from_model = generated_corrections.get((current_index, col), original_text)
            debug_flag = (current_index < SAMPLE_DEBUG_N)
            final_text, accepted = validate_correction(original_text, corrected_text_from_model, col, row.get('id', current_index), debug_print=debug_flag)
            
            if accepted:
                row_copy[col] = final_text
                row_was_changed = True
                corrected_ids.add(row.get('id', current_index))

        processed_rows.append(row_copy.to_dict())
        
        # Lưu tiến trình định kỳ
        if len(processed_rows) % 20 == 0:
            print(f"\\n--- Saving progress at original row index {current_index} ---")
            save_processed_rows_and_exit()
            print("--- Performing periodic memory cleanup ---")
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

except KeyboardInterrupt:
    print("\\nUser interruption detected. Saving processed rows...")
except Exception as e:
    print(f"\\nAn error occurred: {e}")
    print(f"\\nAn unexpected error occurred: {type(e).__name__}: {e}")
finally:
    print("\\nProcessing finished or stopped. Saving any remaining rows...")
    save_processed_rows_and_exit()

    print("\\n--- Summary ---")
    initial_total = len(df_global)
    final_processed_count = 0
    if os.path.exists(OUTPUT_CSV):
        try:
            final_processed_count = len(pd.read_csv(OUTPUT_CSV))
        except Exception: pass
    print(f"Total processed rows in file: {final_processed_count} / {initial_total}.")
    print(f"{len(corrected_ids)} unique IDs were changed in this run.")

    try: del df, df_global, corrected_ids
    except NameError: pass
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
